In [1]:
import os
import sys
import argparse
import logging

import numpy as np
from scipy.stats import norm
from scipy.special import erf
from joblib import Parallel, delayed

sys.path.insert(0, '../../../network')
from network import Population, RateNetwork
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse, ThresholdPlasticityRule
from sequences import GaussianSequence

logging.basicConfig(level=logging.INFO)

In [2]:
def simulate_network(N=40000, c=0.005, S=1, P=30, T=0.3, x_f=1.645, q_f=0.8, seed=42):

    logging.info("Simulating network", x_f)
    phi = ErrorFunction(mu=0.07, sigma=0.05).phi
    exc = Population(N=40000, tau=1e-2, phi=phi)

    sequences = [GaussianSequence(P,exc.size,seed=seed*P+i) for i in range(S)]
    patterns = np.stack([s.inputs for s in sequences])

    conn_EE = SparseConnectivity(source=exc, target=exc, p=0.005, seed=seed)
    plasticity = ThresholdPlasticityRule(x_f=x_f, q_f=q_f)
    synapse = LinearSynapse(conn_EE.K, A=14)
    conn_EE.store_sequences(patterns, synapse.h_EE, plasticity.f, plasticity.g)

    net = RateNetwork(exc, c_EE=conn_EE, formulation=1)

    r0 = exc.phi(plasticity.f(patterns[0,0,:]))
    net.simulate(T, r0=r0)

    state = np.copy(net.exc.state).astype(np.float16)
    return state

In [4]:
N = 40000
c = 0.005
S = 1
P = 30
T = 0.3 

cpu_cores = 8
n_realizations = 1

def run(x_f, q_f, datapath):
    population_rate = np.zeros((n_realizations, len(x_f)))
    for n in range(n_realizations):
        def func(i):
            state = simulate_network(x_f=x_f[i], q_f=q_f[i], seed=n)
            t_max = 250 # np.argmax(overlaps[-1,:])
            return np.mean(state[:,:t_max])
        population_rate[n,:] = Parallel(
            n_jobs=cpu_cores)(delayed(func)(i) for i in range(len(x_f)))

    logging.info("Saving data")
    np.save(open(datapath, "wb"), [x_f, population_rate])

# Varying xf, constraining E(f) to value in panel (a)
x = np.linspace(0.5,0.98,20) 
x_f = norm.ppf(x)
f = lambda x, x_f, q_f: np.where(x < x_f, -(1-q_f), q_f)
avg = norm.expect(lambda x: f(x, 1.645, 0.8))
q_f = 0.5*erf(x_f/np.sqrt(2)) + 0.5 + avg
run(x_f, q_f, "data/data_e1.npy")

# Varying xf, constraining qf==qg
x = np.linspace(0.5,0.98,20)
x_f = norm.ppf(x)
q_f = norm.cdf(x_f)
run(x_f, q_f, "data/data_e2.npy")

INFO:root:Saving data
INFO:root:Saving data
